In [2]:
!pip install langchain langgraph cassio

In [3]:
!pip install langchain_community

In [4]:
!pip install -U tiktoken langchain-groq langchainhub chromadb langchain langgraph langchain_huggingface


    * will use chroma db insted of aster db.
    `step 1: urls = [], load and split the data`

    `step 2: create embeddings using huggingface model`

    `step 3: store embeddings in vector DB`

    ```step 4: from langchain.indexes.vectorstore import VectorStoreIndexWrapper
                astra_vector_store.add_documents(doc_splits)
                print("Inserted %i headlines." % len(doc_splits))

                astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)```

    `step 5: create a retriver`


urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",

    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [7]:
#Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]


#Load 

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

#split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap = 0
)

doc_splits = text_splitter.split_documents(docs_list)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
import os
from langchain_huggingface import HuggingFaceEmbeddings

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\pkhobragade\Desktop\Langgraph\Langgraph\langg\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\pkhobragade\Desktop\Langgraph\Langgraph\langg\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from langchain_community.vectorstores import Chroma


vector_store = Chroma(
    collection_name="langgraph_demo",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

C:\Users\pkhobragade\AppData\Local\Temp\ipykernel_20040\1521201812.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(


In [10]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vector_store.add_documents(doc_splits)
print("Inserted %i headlines." % len(doc_splits))

Inserted 88 headlines.


In [11]:
chroma_vector_index = VectorStoreIndexWrapper(vectorstore=vector_store)

In [12]:
#retriver

retriever = vector_store.as_retriever()

In [13]:
retriever.invoke("What is agent", ConsistencyLevel="LOCAL_ONE")

[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Planning is essentially in order to optimize believability at the moment vs in time.\nPrompt template: {Intro of an agent X}. Here is X\'s plan today in broad strokes: 1)\nRelationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.\nEnvironment info

In [14]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class RouterQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal['vectorstore', 'wiki_search']=Field(
        ...,
        description="Given a user question choose to route it to wikipedia or a vectorstore.",
    )





In [16]:
#lLM

import os 
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")



llm = ChatGroq(model = "Gemma2-9b-It",
            groq_api_key = GROQ_API_KEY)

structured_llm_router = llm.with_structured_output(RouterQuery)

In [17]:
#prompt

system = """
            You are an expert at routing a user question to a vectorstore or Wikipedia.
            The vectorstor contains documents related to Agents, Prompt Engineering and Adversarial Attacks.
            Use the vectorstore for questions on these topics. otherwise, use Wiki-search
            """

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

router_chain = router_prompt | structured_llm_router

print(
    router_chain.invoke(
        {"question" : "What is Project 75 alpha"}
    )
)

print(
    router_chain.invoke(
        {"question": "what is Agent ?"}
)
)

datasource='wiki_search'
datasource='vectorstore'


In [18]:
#creating wiki tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1,
                                    doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [20]:
#Graph

from typing import List
from typing_extensions import TypedDict


class State(TypedDict):
    """ 
    Represent the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: List of documents
    """
    question: str
    generation: str
    documents: List[str]

In [21]:
from langchain.schema import Document

def retriever(state):
    """
    Retrieve documents

    Args:
        state(dict): The current state of graph
    Returns:
        state(dict): New state, that contains retrieved documents
    """
    print("------RETRIEVE------")
    question = state['question']

    #retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question":question}

In [22]:
def wiki_search(state):
    """ 
    wiki search on the re-phrased question.

    Args:
        state (dict): The current state of the graph.
    
    Returns:
        state(dict) : updates documents key wiith appended web result.
    """

    print("------WikiPedia-------")
    question = state['question']
    print(question)

    #wiki
    docs = wiki.invoke({"query": question})
    wiki_result = docs
    wiki_result = Document(page_content=wiki_result)

    return {"documents": wiki_result, "question": question}

In [23]:
### Edges

def route_question(state):
    """ 
    Route question to wiki search or RAG

    Args:
        state(dict): the current state of graph
    
    Return:
        str: Next node to call
    """

    print("-------ROUTE QUESTION--------")
    question = state[question]
    source = router_chain.invoke({"question": question})
    if source.datasource == "wiki_search":
        print("-----ROUTE QUESTION TO Wiki Search-------")
        return "wiki_search"
    elif source.datasource == "vectorstore":
        print("--------ROUTE QUESTION TO RAG--------")
        return "vectorstore"